<a href="https://colab.research.google.com/github/SaturdaysAI-LATAM/Extraccion/blob/master/RecolectorTweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recolector de tweets

Este notebook sirve para recolectar tweets utilizando el API de Twitter.

El propósito es para recolectar datos para el proyecto de Saturdays AI edición LATAM (Equipo Verde).


## Importar librerias

Primero importamos las librerías que vamos a necesitar.

- Tweepy: Wrapper para interactuar con la API de Twitter.
- Pandas: Para crear DataFrames de los datos.
- Tqdm: Es una animación de barra cargando para ver progreso de un loop.


In [ ]:
!pip install tweepy

In [1]:
import os
import tweepy as tw
import pandas as pd
from tqdm import tqdm, notebook

## Autenticación API Twitter

Para comenzar a utilizar tweepy, debemos de crear un objeto de OAuth con nuestras credenciales, estas las podemos obtener en [https://developer.twitter.com/apps](https://developer.twitter.com/apps).

In [2]:
consumer_api_key = "05NZN9fXm1w1pWKNScVZNT1iz"
consumer_api_secret = "IMf59ucljLNOumOOExfTVCX9XQBTTztrYjNwiYeM7lMWnM5gX5"

In [3]:
auth = tw.OAuthHandler(consumer_api_key, consumer_api_secret)

In [4]:
# Objeto API sirve para hacer consultas
api = tw.API(auth, wait_on_rate_limit=True)

## Consultas a la API

### Definición de las consultas

Antes de empezar a consumir datos del API, tenemos que saber qué tweets queremos consumir, para ello se crea un objeto *Cursor* que tiene como atributos los filtros y operadores que necesitamos.

Para filtrar por ciudad, hice un diccionario donde la llave es la ciudad y el valor es `latitud,longitud,radio`, este lo usaremos como atributo en el Cursor.

In [5]:
places = {
    'Aguascalientes': "21.886876,-102.289806,10mi",
    'San Francisco' : "37.781157,-122.398720,10mi",
    'Oklahoma': "35.418006,-97.262495,10mi"
}

Al Cursor le vamos a pasar los filtros para los tweets, los cuales son los siguientes:
- Se excluyen retweets
- Debe contener el término "covid"
- Publicado a partir del 1 de septiembre
- Publicado en un radio de 10 millas de San Francisco
- Tweet está en inglés

Y finalmente el número de tweets que queremos recolectar.

**NOTA:** Si los filtros son muy estrictos, es más probable que tarden los resultados en cargar.


In [6]:
search_words = "-filter:retweets covid"
date_since = "2020-09-01"
place = places["San Francisco"]
total_items = 1000


# Cursor para filtrar tweets
tweets = tw.Cursor(api.search,
              q=search_words,
              geocode=place,
              lang="en",
              since=date_since).items(total_items)

## Obtener los tweets y guardarlos en memoria
Se guardan los tweets en un array y se imprime el progreso.

In [7]:
tweets_copy = []
for tweet in tqdm(tweets):
     tweets_copy.append(tweet)

1000it [00:09, 107.99it/s]


## Construcción del dataset

Los tweets por sí solos vienen con muchos metadatos, aquí se seleccionan los datos más importantes y se insertan en un DataFrame.

Igualmente se muestra el progreso.

In [8]:
tweets_df = pd.DataFrame()
for tweet in tqdm(tweets_copy):
    tweets_df = tweets_df.append(pd.DataFrame({'user_name': tweet.user.name, 
                                               'tweet_id': tweet.id,
                                               'user_location': tweet.user.location,\
                                               'user_description': tweet.user.description,
                                               'user_created': tweet.user.created_at,
                                               'user_followers': tweet.user.followers_count,
                                               'retweets': tweet.retweet_count,
                                               'favorites': tweet.favorite_count,
                                               'user_verified': tweet.user.verified,
                                               'date': tweet.created_at,
                                               'text': tweet.text, 
                                               'source': tweet.source,
                                               'is_quote_status': tweet.is_quote_status,
                                               'is_retweet': tweet.retweeted}, index=[0]))

100%|██████████| 1000/1000 [00:06<00:00, 147.71it/s]


Se ordenan los datos por número de likes y los visualizamos.

In [9]:
tweets_df = tweets_df.sort_values("favorites", ascending=False)

In [10]:
tweets_df.head()

,user_name,tweet_id,user_location,user_description,user_created,user_followers,retweets,favorites,user_verified,date,text,source,is_quote_status,is_retweet
0,Robert Reich,1309946854868529153,"Berkeley, CA","Berkeley prof, frmr Sec of Labor. Cofounder, I...",2010-05-26 23:17:10,1044765,306,665,True,2020-09-26 20:04:02,Covid surging in Wisconsin. Was Trump rally pa...,Twitter for iPhone,False,False
0,Lenny Teytelman,1309601869476503553,"Berkeley, CA","Biologist, cofounder and CEO of @protocolsIO (...",2013-04-10 23:49:48,8096,71,273,False,2020-09-25 21:13:11,Reminder:\n\nYou can find a scientist who does...,Twitter Web App,False,False
0,Bob Wachter,1309861192081399809,San Francisco,"Chair, UCSF Dept of Medicine. Career: What hap...",2011-01-17 02:51:39,113268,108,271,True,2020-09-26 14:23:38,Very informative thread on aerosols by the ess...,Twitter Web App,True,False
0,actually it’s good,1309607187908456448,"San Francisco, CA","talking shit, posting fit",2012-01-20 22:16:22,38900,10,263,False,2020-09-25 21:34:19,yeah I have COVID...\n\nC asual \nO uterwear\n...,Twitter for iPhone,False,False
0,Francisco,1309699913404444672,"San Francisco, CA",🇳🇮🇲🇽,2011-04-05 01:54:59,1442,14,136,False,2020-09-26 03:42:47,Vegas said COVID who? https://t.co/SlcmexlhmV,Twitter for iPhone,False,False


### Creación del csv

Cuando tenemos el DataFrame listo, guardamos los datos en un csv.

La nomenclatura es: `ciudad_fechainicio_fechafin.csv`

In [ ]:
tweets_df.to_csv("covid19_tweets.csv", index=False)